In [ ]:
import sys
import os

# --- 1. Colab環境のクリーンセットアップ ---
if 'google.colab' in sys.modules:
    print("🌐 Colab環境を確認しました。ocp_cad_viewer 用の環境を構築します...")
    
    # 依存関係の競合を避けるため、一旦特定のパッケージを入れ直します
    !pip uninstall -y websockets google-genai ocp-vscode ocp-cad-viewer
    
    # ocp_cad_viewer が必要とする websockets 16.0 を優先してインストール
    !pip install -q "websockets>=16.0,<17.0"
    !pip install -q build123d anthropic openai ipywidgets
    
    # google-genai は最新版を依存関係無視でインストール（これで競合を回避します）
    !pip install -q --no-deps google-genai
    
    # 最後に ocp-cad-viewer をインストール
    !pip install -q ocp-cad-viewer
    
    # --- 2. リポジトリの展開 (ブランチ 06) ---
    BRANCH = "06"
    REPO_URL = "https://github.com/WOCae/Build123d.git"
    
    if not os.path.exists("Build123d"):
        !git clone -b {BRANCH} {REPO_URL}
    
    %cd /content/Build123d
    
    # ソースフォルダへのパスを通す
    src_path = os.path.abspath("src")
    if src_path not in sys.path:
        sys.path.insert(0, src_path)

    try:
        import ocp_cad_viewer
        from build123d import *
        print("✅ 全ての準備が完了しました！ocp_cad_viewer が使用可能です。")
    except ImportError as e:
        print(f"❌ セットアップに失敗しました: {e}")
else:
    print("💻 ローカル環境です。このセルは Colab 専用の修正が含まれています。")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.64.0 requires websockets<15.1.0,>=13.0.0, but you have websockets 16.0 which is incompatible.



  Using cached websockets-16.0-cp310-cp310-win_amd64.whl.metadata (7.0 kB)
Using cached websockets-16.0-cp310-cp310-win_amd64.whl (178 kB)
  Attempting uninstall: websockets
    Found existing installation: websockets 16.0
    Uninstalling websockets-16.0:
      Successfully uninstalled websockets-16.0


ERROR: Could not find a version that satisfies the requirement ocp-cad-viewer (from versions: none)
ERROR: No matching distribution found for ocp-cad-viewer


  Using cached ocp_vscode-3.1.2-py3-none-any.whl.metadata (25 kB)
Using cached ocp_vscode-3.1.2-py3-none-any.whl (684 kB)
  Using cached websockets-15.0.1-cp310-cp310-win_amd64.whl.metadata (7.0 kB)
Using cached websockets-15.0.1-cp310-cp310-win_amd64.whl (176 kB)
  Attempting uninstall: websockets
    Found existing installation: websockets 16.0
    Uninstalling websockets-16.0:
      Successfully uninstalled websockets-16.0
❌ 失敗: No module named 'ocp_cad_viewer'


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ocp-vscode 3.1.2 requires websockets<17.0,>=16.0, but you have websockets 15.0.1 which is incompatible.


In [10]:
import os
import sys

# --- 0. 必要なライブラリを強制インストール ---
!pip install -q build123d anthropic openai google-genai ipywidgets

!pip install -q "websockets>=16.0,<17.0" ocp-cad-viewer
!pip install ocp-cad-viewer -i https://pypi.org/simple
# --- 設定 ---
BRANCH = "05"          # このノートブックがあるブランチ名
SRC_FOLDER = "src"     # build123d_gui.py を入れたフォルダ名
REPO_URL = "https://github.com/WOCae/Build123d.git"

# --- 環境判定とセットアップ ---
if 'google.colab' in sys.modules:
    print("🌐 Colab環境を検出しました。リポジトリを展開します...")
    %cd /content
    
    if not os.path.exists("Build123d"):
        !git clone -b {BRANCH} {REPO_URL}
    
    %cd /content/Build123d
else:
    print("💻 ローカル環境を検出しました。")

# --- ソースフォルダへのパスを通す ---
src_path = os.path.abspath(SRC_FOLDER)
if src_path not in sys.path:
    sys.path.insert(0, src_path)

print(f"✅ 準備完了: {SRC_FOLDER} フォルダ内のモジュールが読み込めるようになりました。")

ERROR: Could not find a version that satisfies the requirement ocp-cad-viewer (from versions: none)
ERROR: No matching distribution found for ocp-cad-viewer


💻 ローカル環境を検出しました。
✅ 準備完了: src フォルダ内のモジュールが読み込めるようになりました。


ERROR: Could not find a version that satisfies the requirement ocp-cad-viewer (from versions: none)
ERROR: No matching distribution found for ocp-cad-viewer


In [8]:
# 外部GUIモジュールのインポート
import build123d_gui

# ダッシュボード表示
build123d_gui.display_dashboard()

ModuleNotFoundError: No module named 'ocp_cad_viewer'

# Build123d 基本動作確認ノートブック

このノートブックでは、Build123dのインストールと基本的な3Dモデル生成を確認します。

**流れ**
1. インストール
2. 基本形状の生成（Box, Cylinder, Sphere）
3. ブーリアン演算（足し算・引き算）
4. フィレット・面取り
5. 機械部品の例（フランジ付きシャフト）
6. ファイル出力（STEP / STL）
7. LLMプロンプトテンプレート

---
> **LLMを使って3Dモデルを生成したい場合は、セル7のプロンプトをコピーして使ってください。**

## 1. インストール

In [ ]:
# build123dのインストール（初回のみ）
import sys
%pip install -q build123d

# Colabの場合、インストール後にランタイム再起動が必要
try:
    from google.colab import runtime
    print('✅ build123d インストール完了')
    print('⚠️  ランタイムを再起動してください（ランタイム → ランタイムを再起動）')
    print('   再起動後はこのセル(セル1)をスキップして、セル2から実行してください。')
except ImportError:
    print('✅ build123d インストール完了（ローカル環境）')


Note: you may need to restart the kernel to use updated packages.
✅ build123d インストール完了（ローカル環境）


In [ ]:
# インポートと動作確認
from build123d import *
import os

print("✅ build123d インポート成功")
print(f"   バージョン確認: Box(10,10,10) → {Box(10,10,10)}")

✅ build123d インポート成功
   バージョン確認: Box(10,10,10) → Box at 0x1bdc326bdf0, label(), #children(0)


## 2. 基本形状の生成

In [ ]:
# ─────────────────────────────────────────────────────────────
# このセルのサンプルコードは 🔬 サンプル タブに移動しました。
# GUIの「🔬 サンプル」タブで ▶ 生成 ボタンを押してください。
# ノートブックの全実行（Run All）では何も生成されません。
# ─────────────────────────────────────────────────────────────


## 3. ブーリアン演算（足し算・引き算）

In [ ]:
# ─────────────────────────────────────────────────────────────
# このセルのサンプルコードは 🔬 サンプル タブに移動しました。
# GUIの「🔬 サンプル」タブで ▶ 生成 ボタンを押してください。
# ノートブックの全実行（Run All）では何も生成されません。
# ─────────────────────────────────────────────────────────────


In [ ]:
# ─────────────────────────────────────────────────────────────
# このセルのサンプルコードは 🔬 サンプル タブに移動しました。
# GUIの「🔬 サンプル」タブで ▶ 生成 ボタンを押してください。
# ノートブックの全実行（Run All）では何も生成されません。
# ─────────────────────────────────────────────────────────────


In [ ]:
# ─────────────────────────────────────────────────────────────
# このセルのサンプルコードは 🔬 サンプル タブに移動しました。
# GUIの「🔬 サンプル」タブで ▶ 生成 ボタンを押してください。
# ノートブックの全実行（Run All）では何も生成されません。
# ─────────────────────────────────────────────────────────────


## 4. フィレット・面取り

In [ ]:
# ─────────────────────────────────────────────────────────────
# このセルのサンプルコードは 🔬 サンプル タブに移動しました。
# GUIの「🔬 サンプル」タブで ▶ 生成 ボタンを押してください。
# ノートブックの全実行（Run All）では何も生成されません。
# ─────────────────────────────────────────────────────────────


## 5. 機械部品の例：フランジ付きシャフト

LLMが生成するコードの典型的な例です。

In [ ]:
# ─────────────────────────────────────────────────────────────
# このセルのサンプルコードは 🔬 サンプル タブに移動しました。
# GUIの「🔬 サンプル」タブで ▶ 生成 ボタンを押してください。
# ノートブックの全実行（Run All）では何も生成されません。
# ─────────────────────────────────────────────────────────────


In [ ]:
# ─────────────────────────────────────────────────────────────
# このセルのサンプルコードは 🔬 サンプル タブに移動しました。
# GUIの「🔬 サンプル」タブで ▶ 生成 ボタンを押してください。
# ノートブックの全実行（Run All）では何も生成されません。
# ─────────────────────────────────────────────────────────────


## 6. ファイル出力（STEP / STL）

- **STEP**: FreeCADやSolidWorksで開ける汎用形式
- **STL**: 3Dプリントや後続のメッシャーへ渡す形式

In [ ]:
# ─────────────────────────────────────────────────────────────
# このセルのサンプルコードは 🔬 サンプル タブに移動しました。
# GUIの「🔬 サンプル」タブで ▶ 生成 ボタンを押してください。
# ノートブックの全実行（Run All）では何も生成されません。
# ─────────────────────────────────────────────────────────────


In [ ]:
# ─────────────────────────────────────────────────────────────
# このセルのサンプルコードは 🔬 サンプル タブに移動しました。
# GUIの「🔬 サンプル」タブで ▶ 生成 ボタンを押してください。
# ノートブックの全実行（Run All）では何も生成されません。
# ─────────────────────────────────────────────────────────────


## (参考) 旧プロンプトテンプレート

APIキーをお持ちでない場合、セル7-Bの `LLM_MODE = 'manual'` を使うと  
同等のプロンプトが自動生成されます。このセクションは参考情報です。


In [ ]:
def show_prompt_template():
    PROMPT_TEMPLATE = """
あなたはBuild123dのエキスパートです。以下のルールに従ってPythonコードを生成してください。

【ルール】
1. 必ず `from build123d import *` でインポートする
2. 形状は `with BuildPart() as part:` のコンテキストマネージャ内に書く
3. 寸法はすべてmm単位のfloatまたはintで書く
4. パラメータは変数として冒頭にまとめて定義する（変更しやすくするため）
5. ブーリアン演算は mode=Mode.SUBTRACT（穴・引き算）または mode=Mode.ADD（合体）を使う
6. スケッチで穴を作る場合は BuildSketch をネストせず、以下のように mode=Mode.SUBTRACT を使う：
   with BuildSketch() as sk:
       Circle(outer_r)
       Circle(inner_r, mode=Mode.SUBTRACT)
7. フィレットは `fillet(part.edges(), radius=R)` で指定する
8. 最後に以下の2行でファイル出力する：
   export_step(part.part, "output.step")
   export_stl(part.part, "output.stl")
9. コード以外の説明文は不要。コードブロックのみ返す。

【基本形状リファレンス】
- Box(length, width, height)
- Cylinder(radius=R, height=H)
- Sphere(radius=R)
- Cone(bottom_radius=R1, top_radius=R2, height=H)

【スケッチ押し出しリファレンス】
- 断面形状を作ってから押し出す方法（パイプ・複雑断面に使う）:
  with BuildSketch() as sk:
      Circle(outer_r)                        # 外形
      Circle(inner_r, mode=Mode.SUBTRACT)    # 内穴（BuildSketchのネストは使わない）
  extrude(amount=length)

【配置・パターン】
- Locations((x, y, z)): 指定座標に移動
- PolarLocations(radius, count): 円周上に均等配置
- GridLocations(x_spacing, y_spacing, x_count, y_count): グリッド配置

【よくある間違い・注意事項】
- BuildSketch を入れ子（ネスト）にしない
- 穴はすべて mode=Mode.SUBTRACT で指定する
- fillet はすべての形状が確定した後、最後に適用する

【作りたいもの】
"""
    print("📋 以下をLLMにコピー＆ペーストして、最後に作りたいものを追記してください：")
    print("="*60)
    print(PROMPT_TEMPLATE)
    print("="*60)
    print("\n例：【作りたいもの】")
    print("外径60mm、内径40mm、長さ100mmのパイプ。両端にR3のフィレットあり。")

## 8. 手動貼り付け実行セル（manualモード用）

`LLM_MODE = 'manual'` でLLMから返ってきたコードをここに貼り付けて実行してください。  
`LLM_MODE = 'api'` の場合はセル7-Bで自動実行されるため、このセルは不要です。


In [ ]:
# ─────────────────────────────────────────────────────────────
# LLMコードの手動実行は 📋 Manual タブに移動しました。
# GUIの「📋 Manual」タブでコードを貼り付けて実行してください。
# ─────────────────────────────────────────────────────────────


---
## 補足メモ

**Colabでの3Dプレビュー**  
Colab上でのインタラクティブな3D表示は現時点では制限があります。  
STLファイルを出力して、別途Three.jsビューワーや外部ツール（MeshLab、FreeCAD等）で確認してください。

**ローカル環境でのプレビュー**  
VS Code + `ocp_vscode` 拡張機能を使うとリアルタイムで3D確認できます。
```
pip install ocp_vscode
```

**後続パーツとの連携**  
- STLファイル → pygmshメッシャーへ渡す  
- STEPファイル → FreeCADやCalculiXのCAEツールへ渡す

**参考リンク**  
- [Build123d 公式ドキュメント](https://build123d.readthedocs.io/)  
- [Build123d GitHub](https://github.com/gumyr/build123d)

In [ ]:
# Manualタブでプロンプトテンプレートを表示したい場合は、
# show_prompt_template() を実行してください。
# 例: show_prompt_template()